**Google Auth**

In [1]:
from google.colab import auth
auth.authenticate_user()

**Setup BigQuery**

In [29]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [4]:
project_id = 'learn-sql-de'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**1. Subqueries/Nested queries**

In [9]:
df_subqueries = client.query('''
SELECT o.order_id, o.user_id,
  (
    SELECT SUM(sale_price)
    FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
    WHERE oi.order_id = o.order_id
    AND oi.status = "Complete"
  ) AS total_sale_price
FROM `bigquery-public-data.thelook_ecommerce.orders` o
  WHERE EXISTS (
    SELECT 1
    FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
    WHERE oi.order_id = o.order_id
    AND oi.status = "Complete"
  ) LIMIT 5;
''').to_dataframe()

In [10]:
df_subqueries.head()

,order_id,user_id,total_order_price
0,2,2,197.979998
1,9,6,39.500000
2,10,7,29.500000
3,18,17,30.209999
4,46,37,109.000000


**2. Join, String manipulaton, aggregate function**

In [13]:
df_joins = client.query('''
SELECT CONCAT(u.first_name, " ", u.last_name) as name,
  SUM(oi.sale_price) as total_sale
FROM `bigquery-public-data.thelook_ecommerce.users` u
JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi ON u.id = oi.user_id
WHERE oi.status = "Complete"
GROUP BY u.id, u.first_name, u.last_name
ORDER BY total_sale DESC
LIMIT 5;
''').to_dataframe()

df_joins.head()

,name,total_sale
0,Ashlee Williams,1359.390000
1,Trevor Spence,1228.000000
2,Timothy Wheeler,1212.880001
3,Todd Weaver,1207.000000
4,Timothy Simpson,1199.450012


This query uses window function `SUM` for **Aggregate Function**, `CONCAT` for **String Manipulation**, and `JOIN`

**3. Date**

In [18]:
df_date = client.query('''
SELECT
 id,
 DATE_DIFF( CURRENT_DATE, CAST(created_at AS DATE), DAY) AS days_since_joined  -- Calculates the difference in days
FROM
 `bigquery-public-data.thelook_ecommerce.users` u
ORDER BY
 days_since_joined DESC
LIMIT 5;
''').to_dataframe()

df_date.head()

,id,days_since_joined
0,79205,2105
1,68141,2105
2,46817,2105
3,22265,2105
4,25739,2105


This data shows the user id with the longest created account.

**4. CTE**

In [23]:
df_cte = client.query('''
WITH user_spending AS (
    SELECT
        u.id,
        CONCAT(u.first_name, ' ', u.last_name) AS name,
        SUM(oi.sale_price) AS total_sale
    FROM
        `bigquery-public-data.thelook_ecommerce.users` u
    JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` oi ON u.id = oi.user_id
    WHERE
        oi.status = 'Complete'
    GROUP BY
        u.id, u.first_name, u.last_name
)
SELECT
    *
FROM
    user_spending
WHERE
    total_sale > 1000
ORDER BY
    total_sale DESC
LIMIT 5;
''').to_dataframe()

df_cte.head()

,id,name,total_sale
0,99909,Ashlee Williams,1359.390000
1,99360,Trevor Spence,1228.000000
2,37917,Timothy Wheeler,1212.880001
3,35238,Todd Weaver,1207.000000
4,55972,Timothy Simpson,1199.450012


This query shows user with the most sale, regarding how many orders

**5. UDF**

In [30]:
%%bigquery df --project learn-sql-de
CREATE TEMP FUNCTION age_category(age INT64)
RETURNS STRING
AS (
  CASE
    WHEN age < 18 THEN 'Minor'
    WHEN age BETWEEN 18 AND 35 THEN 'Young Adult'
    WHEN age BETWEEN 36 AND 55 THEN 'Middle Aged'
    ELSE 'Senior'
  END
);

SELECT
    id,
    first_name,
    last_name,
    age,
    age_category(age) AS age_group  -- No need for dataset name, since it's a temp function
FROM
    `bigquery-public-data.thelook_ecommerce.users`
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

**6. Case Statement**

In [31]:
df_case = client.query('''
SELECT
    id,
    name,
    retail_price,
    CASE
        WHEN retail_price < 50 THEN 'Budget'
        WHEN retail_price BETWEEN 50 AND 200 THEN 'Standard'
        ELSE 'Premium'
    END AS price_category
FROM
    `bigquery-public-data.thelook_ecommerce.products`
ORDER BY
    retail_price DESC;
''').to_dataframe()

df_case.head()

,id,name,retail_price,price_category
0,23546,Alpha Industries Rip Stop Short,999.0,Premium
1,24447,Darla,999.0,Premium
2,24110,Woolrich Arctic Parka DF,990.0,Premium
3,24341,Nobis Yatesy Parka,950.0,Premium
4,2796,ASCIS Cushion Low Socks (Pack of 3),903.0,Premium


**7. Pivot**

In [36]:
df_pivot = client.query('''
SELECT
    user_id,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 1 THEN sale_price ELSE 0 END) AS january_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 2 THEN sale_price ELSE 0 END) AS february_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 3 THEN sale_price ELSE 0 END) AS march_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 4 THEN sale_price ELSE 0 END) AS april_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 5 THEN sale_price ELSE 0 END) AS may_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 6 THEN sale_price ELSE 0 END) AS june_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 7 THEN sale_price ELSE 0 END) AS july_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 8 THEN sale_price ELSE 0 END) AS august_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 9 THEN sale_price ELSE 0 END) AS september_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 10 THEN sale_price ELSE 0 END) AS october_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 11 THEN sale_price ELSE 0 END) AS november_sales,
    SUM(CASE WHEN EXTRACT(MONTH FROM created_at) = 12 THEN sale_price ELSE 0 END) AS december_sales
FROM
    `bigquery-public-data.thelook_ecommerce.order_items`
WHERE
    status = 'Complete'
GROUP BY
    user_id
LIMIT 5;
''').to_dataframe()

# Now you can call head() to see the result
df_pivot.head()


,user_id,january_sales,february_sales,march_sales,april_sales,may_sales,june_sales,july_sales,august_sales,september_sales,october_sales,november_sales,december_sales
0,4689,0.0,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.000000,0.0
1,45936,0.0,15.77,0.0,0.0,0.0,35.18,0.0,0.0,0.0,0.0,0.000000,0.0
2,13309,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,1.5
3,42841,0.0,1.50,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0
4,11924,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,286.360002,0.0


This query shows sales per user monthly

**8. Recursive**

In [47]:
df_recursive = client.query('''
WITH RECURSIVE event_hierarchy AS (
    -- Base case: Select the first event
    SELECT
        id,
        user_id,
        created_at,
        1 AS level
    FROM
        `bigquery-public-data.thelook_ecommerce.events`
    WHERE
        created_at = (SELECT MIN(created_at) FROM `bigquery-public-data.thelook_ecommerce.events`)

    UNION ALL

    SELECT
        e.id,
        e.user_id,
        e.created_at,
        eh.level + 1
    FROM
        `bigquery-public-data.thelook_ecommerce.events` e
    JOIN
        event_hierarchy eh ON e.user_id = eh.user_id
    WHERE
        e.created_at > eh.created_at
)

SELECT
    *
FROM
    event_hierarchy
ORDER BY
    level, created_at
LIMIT 10;
''').to_dataframe()

df_recursive.head()


,id,user_id,created_at,level
0,1902071,<NA>,2019-01-02 00:08:00+00:00,1
1,1902070,<NA>,2019-01-02 00:08:00+00:00,1


**9. Window Functions**

In [43]:
df_window = client.query('''
SELECT
    oi.user_id,
    u.first_name,
    u.last_name,
    oi.order_id,
    oi.sale_price,
    SUM(oi.sale_price) OVER (PARTITION BY oi.user_id ORDER BY oi.created_at) AS running_total_sales
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
    `bigquery-public-data.thelook_ecommerce.users` u ON oi.user_id = u.id
WHERE
    oi.status = 'Complete'
ORDER BY
    oi.user_id, oi.created_at;
''').to_dataframe()

df_window.head()

,user_id,first_name,last_name,order_id,sale_price,running_total_sales
0,2,Melissa,Gonzalez,2,79.000000,79.000000
1,2,Melissa,Gonzalez,2,25.000000,104.000000
2,2,Melissa,Gonzalez,2,67.989998,171.989998
3,2,Melissa,Gonzalez,2,25.990000,197.979998
4,5,Alex,Taylor,6,125.000000,125.000000


**10. Union**

In [44]:
df_union = client.query('''
SELECT
    oi.order_id AS source_id,
    oi.user_id,
    oi.product_id,
    oi.sale_price AS price,
    'Order Item' AS source
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` oi
WHERE
    oi.status = 'Complete'

UNION ALL

SELECT
    p.id AS source_id,
    NULL AS user_id,
    p.id AS product_id,
    p.retail_price AS price,
    'Product' AS source
FROM
    `bigquery-public-data.thelook_ecommerce.products` p
WHERE
    p.retail_price IS NOT NULL;
''').to_dataframe()

df_union.head()

,source_id,user_id,product_id,price,source
0,13842,<NA>,13842,6.25,Product
1,13928,<NA>,13928,5.95,Product
2,14115,<NA>,14115,10.99,Product
3,14157,<NA>,14157,10.99,Product
4,14273,<NA>,14273,15.99,Product
